In [ ]:
!pip install --upgrade google-api-python-client pandas numpy nltk vaderSentiment textblob transformers matplotlib seaborn scikit-learn sqlalchemy


In [1]:
!pip install vaderSentiment

In [2]:
from googleapiclient.discovery import build
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import re


In [ ]:
import re
video_url = input("Enter YouTube Study Video URL: ")
def extract_video_id(url):
   video_id = extract_video_id(video_url)
   pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
   match = re.search(pattern, url)
   return match.group(1) if match else None

video_id = extract_video_id(video_url)

if video_id is None:
    raise ValueError("Invalid YouTube URL. Unable to extract video ID.")


In [ ]:

#import os

#API_KEY = os.getenv("YOUTUBE_API_KEY")

#if not API_KEY:
    #raise RuntimeError("YOUTUBE_API_KEY not found")

youtube = build(
    "youtube",
    "v3",
    developerKey=API_KEY
)


In [4]:
def fetch_video_metadata(video_id):
    request = youtube.videos().list(
        part="snippet,statistics",
        id=video_id
    )
    response = request.execute()

    if not response["items"]:
        return None

    item = response["items"][0]

    return {
        "title": item["snippet"]["title"],
        "views": int(item["statistics"].get("viewCount", 0)),
        "likes": int(item["statistics"].get("likeCount", 0)),
        "comments": int(item["statistics"].get("commentCount", 0))
    }


In [11]:
from datetime import datetime, timezone

def days_since_upload(published_at):
    published = datetime.fromisoformat(published_at.replace("Z", "+00:00"))
    return max((datetime.now(timezone.utc) - published).days, 1)


In [5]:
def fetch_comments(video_id, max_comments=100):
    comments = []
    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            textFormat="plainText"
        )
        response = request.execute()
        for item in response.get("items", []):
            comments.append(
                item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            )
    except:
        pass
    return comments


In [6]:
analyzer = SentimentIntensityAnalyzer()

def compute_sentiment(comments):
    if not comments:
        return 0

    scores = [analyzer.polarity_scores(c)["compound"] for c in comments]
    return np.mean(scores)


In [12]:
def compute_effectiveness(metadata, avg_sentiment):
    views = metadata["views"]
    likes = metadata["likes"]
    comments = metadata["comments"]
    days = metadata["days"]

    # Daily rates
    views_per_day = views / days
    likes_per_day = likes / days
    comments_per_day = comments / days

    # Learning engagement rate
    engagement_rate = likes_per_day + 2 * comments_per_day

    # Normalize components
    view_score = min((views_per_day / 5000) * 30, 30)
    engagement_score = min((engagement_rate / 50) * 50, 50)
    sentiment_score = (avg_sentiment + 1) * 10  # 0–20

    final_score = view_score + engagement_score + sentiment_score
    return round(final_score, 2)


In [14]:
def classify_video(score, views, comments):
    if views < 500 or comments < 5:
        return "Low Effectiveness"

    if final_score >= 70:
        return "Highly Effective"
    elif final_score_score >= 40:
        return "Moderately Effective"
    else:
        return "Low Effectiveness"

In [9]:


metadata = fetch_video_metadata(video_id)
comments = fetch_comments(video_id)
avg_sentiment = compute_sentiment(comments)

final_score = compute_effectiveness(metadata, avg_sentiment)
label = classify_video(final_score)

print("\n==============================")
print("YouTube Study Video Analysis")
print("==============================")
print("Title:", metadata["title"])
print("Views:", metadata["views"])
print("Likes:", metadata["likes"])
print("Comments:", metadata["comments"])
print("Average Sentiment:", round(avg_sentiment, 3))
print("Effectiveness Score:", round(final_score, 2))
print("Result:", label)
print("==============================")
if metadata["views"] < 50:
    print("Video too new or low-viewed; score may be unreliable")



TypeError: classify_video() missing 2 required positional arguments: 'views' and 'comments'